# Remote Work vs Travel Data

In [1]:
# Imports:
import pprint, json, csv, os.path as path
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import requests as rq

# Add a config.py file and add a line that says 
# bls_api_key = "your key here"
# ita_api_key = "your key here"
# replace "your key here with your BLS api key"
from config import bls_api_key, ita_api_key


In [6]:
bls_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

# this does not work, trying sample code
# outp = rq.post(bls_url).json()

# pprint(outp)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [1]:
# SAMPLE CODE PROVIDED BY BLS

import requests
import json
import prettytable
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0','SUUR0000SA0'],"startyear":"2011", "endyear":"2014"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x.get_string())
    output.close()